[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openai/openai-cookbook/blob/main/articles/gpt-oss/run-colab.ipynb)

# Run OpenAI gpt-oss 20B in a FREE Google Colab

OpenAI released `gpt-oss` [120B](https://hf.co/openai/gpt-oss-120b) and [20B](https://hf.co/openai/gpt-oss-20b). Both models are Apache 2.0 licensed.

Specifically, `gpt-oss-20b` was made for lower latency and local or specialized use cases (21B parameters with 3.6B active parameters).

Since the models were trained in native MXFP4 quantization it makes it easy to run the 20B even in resource constrained environments like Google Colab.

Authored by: [Pedro](https://huggingface.co/pcuenq) and [VB](https://huggingface.co/reach-vb)

## Setup environment

Since support for mxfp4 in transformers is bleeding edge, we need a recent version of PyTorch and CUDA, in order to be able to install the `mxfp4` triton kernels.

We also need to install transformers from source, and we uninstall `torchvision` and `torchaudio` to remove dependency conflicts.

In [ ]:
!pip install -q --upgrade torch==2.8

In [ ]:
!pip install -q transformers triton==3.4 kernels

In [ ]:
!pip uninstall -q torchvision torchaudio -y

Please, restart your Colab runtime session after installing the packages above.

## Load the model from Hugging Face in Google Colab

We load the model from here: [openai/gpt-oss-20b](https://hf.co/openai/gpt-oss-20b)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

## Specify Reasoning Effort

Simply pass it as an additional argument to `apply_chat_template()`. Supported values are `"low"`, `"medium"` (default), or `"high"`.

In [ ]:
guidelines= '''You will be given a text.
If there are two samples, the first is English and the second is Irish. If you are given one, the text is Irish.
Using the guidelines, give the correct code for the text: NL, WL, X, CS, CB, or CC. Finish your statement in the format 'Annotation: [value]';


Guidelines: IF the content is NOT linguistic OR you are unsure THEN annotate as NL NOTE: ""No or unsure"" → NL ELSE IF the text is parallel data AND the target side is NOT in Modern (standardised) Irish OR the source side is NOT in Modern English THEN annotate as WL NOTE: ""No or Unsure"" → WL NOTE: Untranslated named entities on either side are treated as belonging to that language (e.g., ""Capnat"" on the English side counts as English) NOTE: If text is entirely named entities, it is more likely to be ‘Yes’ ELSE IF the Irish target text is NOT a direct translation of the English source text THEN annotate as X NOTE: ""Yes or unsure"" → next question; if not, annotate X ELSE IF the Irish text is short (just headings, single unrelated phrases, OR five words or fewer) THEN annotate as CS NOTE: Short text = CS ELSE IF the text is boilerplate OR low quality (text that would repeat across similar webpages, unnatural code-switching, unnatural phrasing, frequent unnatural formatting, misalignments) THEN annotate as CB NOTE: CB includes text not representative of natural language One formatting problem alone is NOT enough; more than one may qualify ELSE annotate as CC NOTE: CC = natural language, not short, not boilerplate, properly aligned.'''


texts = ['"", "Tarluithe Diat Worms Tógadh Taigh BhroughtonI gCille Chùithbeirt, Alba. Breitheanna 19 Feabhra — Luigi Boccherini, cumadóir ceoil is dordveidhleadóir Iodálach 24 Feabhra — Joseph Banks, luibheolaí Sasanach (b. 1820) 13 Aibreán — Thomas Jefferson 3ú uachtarán na Stát Aontaithe Mheiriceá (b. 1826) 24 Aibreán — Edmund Cartwright, ceapadóir Sasanach (b. 1823) 20 Bealtaine — Toussaint Louverture, ceannaire réabhlóideach Háitíoch (b. 1803) 24 Bealtaine — Jean-Paul Marat, idé-eolaí agus tíoránach cruálach le linn Réabhlóid na Fraince 26 Lúnasa — Antoine Lavoisier, ceimiceoir Francach Timpeall na bliana seo – Eibhlín Dubh Ní Chonaill, file Éireannach (b. 1800) 1 Nollaig- Martin Heinrich Klaproth, poitigéir Gearmánach Básanna 2 Iúil  — Spencer Compton, príomh-aire na Breataine Blianta"',
         '"Healthcare Business Valuations", "dtugtar Cailín Kim watch gan"',
         '"Within the ceiling defined above, it shall calculate the amount to be actually levied in such a way that the Government subsidies to be actually paid shall be at least equal to that amount;", "Laistigh den uasteorainn ala sainithe thuas, déanfaidh sé méid an tobhaigh iarbhir a riomh ar dhóigh go mbeidh na fóirdheontais Rialtais a íocfar iarbhír cóímhéid leis an tobhach sin, ar an gcuid is lú de."']


for text in texts:
  messages = [
      {"role": "system", "content": guidelines},
      {"role": "user", "content": text},
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt",
      return_dict=True,
      reasoning_effort="low",
  ).to(model.device)

  generated = model.generate(**inputs, max_new_tokens=500)
  print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

## Try out other prompts and ideas!

Check out our blogpost for other ideas: [https://hf.co/blog/welcome-openai-gpt-oss](https://hf.co/blog/welcome-openai-gpt-oss)